### * task 내용
* 라이브러리, 임베딩 모델 설정 자유, 검색 응답 속도 0.5sec 미만
* semantic 검색 자체의 의미, 라이브러리 선정 사유 및 동작 원리

###### --------------------------------------------------------------------------------------------------------------------------------

# semantic search
* seantic search는 사용자 검색의도를 파악하고, 문서에 기술된 어휘의 의미 문맥을 분석하여, 사용자가 원하는 검색 결과를 제시하는 방법

### 1. 라이브러리 선정 사유 및 모델 구조
* sentence-bert 선정 사유는 bert의 임베딩의 성능을 우수하게 개선시킨 모델
* BERT로 부터 문장 백터를 얻는 방법
* - BERT의 [CLS] 토큰의 출력 벡터를 문장 벡터로 간주한다.
* - BERT의 모든 단어의 출력 벡터에 대해서 평균 풀링을 수행한 벡터를 문장 벡터로 간주한다.

* 특히 KorSTS 학습데이터로 학습하였기 때문에 다른 sbert에 비해 한국어에 특화되어 있다고 판단하였다.
* KorSTS는 두 문장 사이의 유사도를 점수로 평가한 데이터셋이다. 
* 구조 a,b를 입력 받아서 768차원으로 백터로 변환하고, -> 평균풀링 레이어를 사용하여 특징을 추출하고, 그 두 문장의 백터의 유사도를 코사인 유사도로 변경 하여 계산

###### --------------------------------------------------------------------------------------------------------------------------------

* KorSTS dataset 예시

* - main-captions |	MSRvid | 2012test |	0001 |	5.000 |	비행기가 이륙하고 있다.           |	비행기가 이륙하고 있다.
* - main-captions |	MSRvid | 2012test |	0004 |	3.800 |	한 남자가 큰 플루트를 연주하고 있다. |	남자가 플루트를 연주하고 있다.

In [1]:
# 모델설치
# !pip install ko-sentence-transformers

* 데이터 불러오기
* 약 40만개

In [3]:
# 파일 불러오기 contents 딕셔너리에서 꺼낸값
import pickle

with open('/Users/seunghyun/anaconda3/envs/my_env/23.09.14/data.pkl', 'rb') as file:
    data = pickle.load(file)

#전처리
import re
# 기호와 기호안에 들어간 영어 제거
data['contents'] = data['contents'].apply(lambda x: re.sub(r'[^가-힣\s]', '', x))
data['add_content'] = data['title_x'] + data['contents']

In [20]:
len(data)

458339

In [21]:
sentences = data['add_content'].tolist()

* https://huggingface.co/jhgan/ko-sroberta-multitask ko-sroberta-multitask 평가 84.77%
* https://huggingface.co/jhgan/ko-sbert-sts ko-sbert-sts 평가 81.55%

In [44]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import time

embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

embedding_start = time.time()
# 문장을 tensor 형태로 변환
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True)
embedding_end = time.time()
embedding_time = embedding_end - embedding_start
print(f"임베딩 시간 {embedding_time/60/60} 시간")      

임베딩 시간 2.4454372345076667 시간


In [45]:
# 임베딩 값 저장
with open('/Users/seunghyun/anaconda3/envs/my_env/23.09.14/semantic_search_embedding_sroberta.pkl', 'wb') as f:
     pickle.dump(corpus_embeddings, f)

In [10]:
# sroberta
from sentence_transformers import SentenceTransformer, util
import pickle
import numpy as np
# 파일 불러오기 contents 딕셔너리에서 꺼낸값

with open('/Users/seunghyun/anaconda3/envs/my_env/23.09.14/data.pkl', 'rb') as file:
    data = pickle.load(file)

#전처리
import re
# 기호와 기호안에 들어간 영어 제거
data['contents'] = data['contents'].apply(lambda x: re.sub(r'[^가-힣\s]', '', x))
data['add_content'] = data['title_x'] + data['contents']

# 임베딩 값 불러오기
with open("/Users/seunghyun/anaconda3/envs/my_env/23.09.14/semantic_search_embedding_sroberta.pkl", 'rb') as f:
     corpus_embeddings = pickle.load(f)
# 문장 불러오기
sentences = data['add_content'].tolist()
# 검색
import time
queries = input('검색 하실 내용을 입력하시오 : ')
queries = [queries]
embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 5개만 출력
top_k = 10

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    
    start_time = time.time()
    # input 문장과, 말뭉치 embedding값을 코사인 유사도로 비교
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    # 순위를 순서대로 맞추기 위함
    # np.argpartition는 큰값 5개를 뽑아 오른쪽으로 넣는 함수
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k] 
    
    end_time = time.time()

    print("\n\n======================\n\n")
    print("검색어: ", query)
    print(f"검색결과 {top_k}개 출력")
    print("\n\n======================\n\n")
    # score가 높은 top5개를 뽑고 score도 부여한다.
    for index, idx in enumerate(top_results[0:top_k]):
        print(f"{index+1}번쨰 :",sentences[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))
        
    print(" ")
    search_time = end_time - start_time
    print(f"검색 시간: {search_time:.4f} 초")





검색어:  제가 음식을 심키는데 다른?구멍으로 넘어간것같은 느낌이 나고 이물감 약간 느껴지는거 빼곤 별다른 건 없는데 괜찮은거죠??
검색결과 10개 출력




1번쨰 : 음식먹을때 가끔시큰거리는 현상왼쪽 정확히 어딘지는 모르겠으나 어금니쪽에 김치등 질긴 음식을 먹을때만 가끔 시큰거리는 느낌이 있는데 오히려 딱딱한 음식먹을때는 괜찮습니다 (Score: 0.7464)
2번쨰 : 배달음식을 먹는도중에 이물질이?배달음식제육뽁음을 먹는도중에 무언가가발견되었는데 세상에나첨엔 무언가 싶었는데자세히 보니 임플란트 였네요 저는 임플란트를 하지않아 제건 아닌게 분명하고식당사장님께 사진 보내드리고 말씀드렸는데사장님도 황당해 하셨어요 왜냐면 그 식당이관리가 철저하거든요   더군다나 머리카락이나다른이물질이라면 이해가 가겠지만 저도 많이당황스럽네요  배탈이 난것도 아니고 직접 씹은것도아니라 이상은  없지만 상당히 불쾌하고 기분이좋지 않습니다  그냥 넘어가자니 그건 아닌것 같고어떻게 해야 할까요 전문가님들의 고견을 듣고싶습니다 (Score: 0.7226)
3번쨰 : 상한음식 조금 먹었는데 괜찮을까요??상한음식 모르고 한숟갈 먹었는데요 분 지났어요 아직까지 아무 렇지는 않은데조금 정도는 상관없을까요 괜히 걱정되네요 (Score: 0.7091)
4번쨰 : 기도에 작음 음식물이 넘어간 것 같습니다.햄을 잘게 잘라먹다 후두 쪽으로 음식물이 넘어간 것 같습니다보통 음식물이 넘어가면 사례가 나오긴 하는데 최근들에 물을 잘못 삼킨 적도 많지만저는 음식을 잘못 삼켜도 기침이 잘 나오질 않더라구요 아주 얕게 기침이 나오는 정도고 그 이후론 잠잠합니다삼킨 기분 탓 인것 치고는 기도와 후두 부근에서 이물감이 너무나도 선명하게 들고삼킨 이후부터 목 통증과 가슴 호흡이 묘하게 답답하기까지 합니다하임리이법이라도 시행하면 괜찮아 질까요 (Score: 0.7000)
5번쨰 : 임플란트 시술후 음식을 먹을때어금니 임플란트 시술후 음식을 먹을때 두꺼운 종이를 씹는듯한 느낌입니다 임플란트를 한 아금니 아래 위 